# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Study data files
file_loc = "../WeatherPy/output_data/City_Data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(file_loc)

weather_data.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Geraldton,-28.77,114.60,68.00,45,75,18.34,AU,1588751668
1,Halifax,44.65,-63.57,37.00,73,95,3.00,CA,1588751386
2,Torbay,47.67,-52.73,35.01,92,90,11.41,CA,1588751668
3,Nikolskoye,59.70,30.79,61.00,38,23,8.95,RU,1588751541
4,Busselton,-33.65,115.33,57.99,56,86,17.65,AU,1588751669


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
location = weather_data[["Latitude", "Longitude"]].astype(float)

# Fill NaN values and convert to float
humidity = weather_data["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

#save figure
plt.savefig("output_data/vacationheatmap.png")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
criteria_1 = weather_data.loc[(weather_data["Max Temp"]>70) & (weather_data["Max Temp"]<80),:]

# #Wind speed less than 10 mph.
criteria_2 = criteria_1.loc[criteria_1["Wind Speed"]<10,:]

# #Zero cloudiness.
criteria_3 = criteria_2.loc[criteria_2["Cloudiness"]==0,:]

criteria_3.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,Cape Town,-33.93,18.42,73.00,72,0,9.17,ZA,1588751509
114,Hervey Bay,-25.30,152.85,71.01,62,0,5.01,AU,1588751693
169,Awjilah,29.11,21.29,75.97,26,0,5.77,LY,1588751707
233,Gazli,40.13,63.45,76.98,31,0,4.16,UZ,1588751721
340,Qena,26.16,32.72,77.00,25,0,9.17,EG,1588751745


In [7]:
criteria_3.dropna(inplace=True)

/Users/HyunsooKim/opt/anaconda3/envs/gmaps/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# from pprint import pprint

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "location": "-33.93, 18.42", 
#     "radius": "5000",
#     "type": "lodging",
#     "key": g_key,
# }

# response = requests.get(base_url, params=params).json()

# pprint(response)

# result[0]["name"]

{'html_attributions': [],
 'next_page_token': 'CqQCHAEAAI0AECwZptdG-INtUnjpvvdGmxUPbggAUTtnrInzmm-cU6-lRplMZn7rP38WohJpbgupL0UsVYiV5d9ZgJOE5B6vHnzBomKj1n6aEIcYq7_bNZrmjj2jkI6V3doKMIOTgVzzsH8uF064Pk4rDKWR9Wn1yiIqdM4eh_t__X-2F0Y5EykM-Wy3soP6kbTybuXGYDidmsd7SWKkZOWol-kl4ygq2F07qhrkwz-gGVU1-jqvkRkb8rncy2ONbPw6A0ldJpoHEYFYzkv6s0qQNJK-VOMnMvllmeCeKgYjlUlOMyNFINGePodZc_WKz3vt26112WJIGlF_Vq7AEtMf5mJ1dPiZiVrxSspU-Sp7popf4v_pWy68s3UHflNS1rUFDs7B_hIQp6leV4JX7pStlmLyGis2UBoU4qcaHixdkYF7Rcy438U2rdbiBAE',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -33.9157633, 'lng': 18.4234956},
                           'viewport': {'northeast': {'lat': -33.9143788197085,
                                                      'lng': 18.42475433029151},
                                        'southwest': {'lat': -33.9170767802915,
                                                      'lng': 18.4220563697085}}},
              'icon': 'https://maps.gstatic.com/mapf

NameError: name 'result' is not defined

In [9]:
hotel_df = criteria_3.copy()

# set up additional columns to hold information
hotel_df['Hotel Name'] = ""

hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,Cape Town,-33.93,18.42,73.00,72,0,9.17,ZA,1588751509,
114,Hervey Bay,-25.30,152.85,71.01,62,0,5.01,AU,1588751693,
169,Awjilah,29.11,21.29,75.97,26,0,5.77,LY,1588751707,
233,Gazli,40.13,63.45,76.98,31,0,4.16,UZ,1588751721,
340,Qena,26.16,32.72,77.00,25,0,9.17,EG,1588751745,


In [12]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
}

response = requests.get(base_url, params=params).json()

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get locations of the city
    loc_lat=row["Latitude"]
    loc_lon=row["Longitude"]
    locat=f"{loc_lat},{loc_lon}"
    cit = row["City"]

    # add keyword to params dict
    params['location'] = locat

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {cit}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
   
   
    try:
        hotel = results[0]["name"]

        print(f"Closest hotel to {cit} at {loc_lat}, {loc_lon} is {hotel}.")
        
        hotel_df.loc[index, 'Hotel Name'] = hotel

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 15: Cape Town.
Closest hotel to Cape Town at -33.93, 18.42 is Southern Sun Waterfront Cape Town.
------------
Retrieving Results for Index 114: Hervey Bay.
Closest hotel to Hervey Bay at -25.3, 152.85 is Shelly Bay Resort.
------------
Retrieving Results for Index 169: Awjilah.
Missing field/result... skipping.
------------
Retrieving Results for Index 233: Gazli.
Missing field/result... skipping.
------------
Retrieving Results for Index 340: Qena.
Closest hotel to Qena at 26.16, 32.72 is Grand hotel "Quena".
------------
Retrieving Results for Index 387: Vredendal.
Closest hotel to Vredendal at -31.67, 18.5 is Tharrakamma Guest House.
------------
Retrieving Results for Index 415: Saýat.
Missing field/result... skipping.
------------
Retrieving Results for Index 457: Nerang.
Closest hotel to Nerang at -27.98, 153.33 is Town & Country Motel & Restaurant.
------------
Retrieving Results for Index 488: Tshane.
Missing field/result... skipping.
------------
R

In [13]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,Cape Town,-33.93,18.42,73.00,72,0,9.17,ZA,1588751509,Southern Sun Waterfront Cape Town
114,Hervey Bay,-25.30,152.85,71.01,62,0,5.01,AU,1588751693,Shelly Bay Resort
169,Awjilah,29.11,21.29,75.97,26,0,5.77,LY,1588751707,Awjilah
233,Gazli,40.13,63.45,76.98,31,0,4.16,UZ,1588751721,Gazli
340,Qena,26.16,32.72,77.00,25,0,9.17,EG,1588751745,"Grand hotel ""Quena"""
387,Vredendal,-31.67,18.50,76.01,23,0,8.23,ZA,1588751755,Tharrakamma Guest House
415,Saýat,38.78,63.88,78.80,44,0,4.70,TM,1588751761,Sayat
457,Nerang,-27.98,153.33,71.01,68,0,3.00,AU,1588751771,Town & Country Motel & Restaurant
488,Tshane,-24.02,21.87,72.14,31,0,9.08,BW,1588751779,Tshane
492,Ixtapa,20.70,-105.20,73.99,84,0,1.01,MX,1588751637,Container Inn Puerto Vallarta


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map

heat_layer = gmaps.heatmap_layer(location, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=100,
                                 point_radius = 1, 
                                 opacity=1)

marker_layer = gmaps.marker_layer(locations, 
                                  hover_text='', 
                                  label='', 
                                  info_box_content=hotel_info)


fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

plt.savefig("output_data/vacation_with_hotels.png")


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>